In [82]:
import pandas as pd
import scipy.stats as st
import re
import matplotlib.pyplot as plt
import seaborn as sns

# Задание 1.

### 1)

In [83]:
game_sales = pd.read_csv('vgsales.csv', na_values='NaN')
game_sales.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
sport_games = game_sales[game_sales.Genre == 'Sports']
sport_games.shape

(2348, 16)

Скажем, что критики относятся к игре хорошо, если их средняя оценка больше 60 (по аналогии с пятибальной шкалой, где 4 это хорошая оценка). В противном случае утверждаем обратное.
- Н0 - Критики не любят спортивные игры (<= 60)
- Н1 - Критики хорошо относятся к спортивным играм (> 60)

In [85]:
border = 60

Данных много, примем уровень значимости равным 0,01:

In [86]:
alpha = 0.01

Применим односторонний Т-критерий Стьюдента:

In [87]:
t, p = st.ttest_1samp(sport_games['Critic_Score'].dropna(), border, alternative='greater')
print('t-критерий:', t)
print('p-value:', p)
if p < alpha:
    print('Принимаем гипотезу Н1, отвергаем гипотезу Н0')
else:
    print('Не отвергаем гипотезу Н0')

t-критерий: 29.79817370420808
p-value: 1.3251047326077337e-146
Принимаем гипотезу Н1, отвергаем гипотезу Н0


In [88]:
sport_games['Critic_Score'].mean()

71.96817420435511

Тест Стьюдента обнаружил статистически значимые результаты, что говорит о хорошем отношении критиков к видеоиграм на спортивную тематику.

### 2)

In [89]:
# На PS4
ps4_games = game_sales[game_sales.Platform == 'PS4']
display(ps4_games.head())
print(ps4_games.shape)

# На РС
pc_games = game_sales[game_sales.Platform == 'PC']
display(pc_games.head())
print(pc_games.shape)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
31,Call of Duty: Black Ops 3,PS4,2015.0,Shooter,Activision,6.03,5.86,0.36,2.38,14.63,NaN,NaN,NaN,NaN,NaN,NaN
42,Grand Theft Auto V,PS4,2014.0,Action,Take-Two Interactive,3.96,6.31,0.38,1.97,12.61,97.0,66.0,8.3,2899.0,Rockstar North,M
77,FIFA 16,PS4,2015.0,Sports,Electronic Arts,1.12,6.12,0.06,1.28,8.57,82.0,42.0,4.3,896.0,EA Sports,E
87,Star Wars Battlefront (2015),PS4,2015.0,Shooter,Electronic Arts,2.99,3.49,0.22,1.28,7.98,NaN,NaN,NaN,NaN,NaN,NaN
92,Call of Duty: Advanced Warfare,PS4,2014.0,Shooter,Activision,2.81,3.48,0.14,1.23,7.66,83.0,39.0,5.7,1443.0,Sledgehammer Games,M


(393, 16)


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
85,The Sims 3,PC,2009.0,Simulation,Electronic Arts,0.99,6.42,0.0,0.60,8.01,86.0,75.0,7.6,886.0,The Sims Studio,T
138,World of Warcraft,PC,2004.0,Role-Playing,Activision,0.08,6.21,0.0,0.00,6.29,93.0,57.0,7.3,2182.0,Blizzard Entertainment,T
192,Diablo III,PC,2012.0,Role-Playing,Activision,2.44,2.16,0.0,0.54,5.14,88.0,86.0,4,9629.0,Blizzard Entertainment,M
195,Microsoft Flight Simulator,PC,1996.0,Simulation,Microsoft Game Studios,3.22,1.69,0.0,0.20,5.12,NaN,NaN,NaN,NaN,NaN,NaN
218,StarCraft II: Wings of Liberty,PC,2010.0,Strategy,Activision,2.57,1.68,0.0,0.58,4.84,93.0,82.0,8.2,3052.0,Blizzard Entertainment,T


(974, 16)


В данном случае стоит задача определить на какой платформе критикам игры нравятся больше. Будем поступать следующим образом: зададимся двумя гипотезами, по которым будем судить, есть ли симпатия критиков к одной из платформ или же критики относятся одинаково к обеим из них; далее, если будет перевес в чью-либо сторону, по знаку перед критерием Стьюдента мы скажем, к какой платформе симпатия больше. Приступаем:
- Н0 - отношение критиков одинаковое
- Н1 - отношение критиков разное

Данных много, уровень значимости идентичен п.1.

In [90]:
alpha = 0.01

In [91]:
t, p = st.ttest_ind(ps4_games['Critic_Score'].dropna(), pc_games['Critic_Score'].dropna(), equal_var=False)
print('t-критерий:', t)
print('p-value:', p)
if p < alpha:
    print('Принимаем гипотезу Н1, отвергаем гипотезу Н0')
else:
    print('Не отвергаем гипотезу Н0')

t-критерий: -4.3087588262138725
p-value: 2.067249157283479e-05
Принимаем гипотезу Н1, отвергаем гипотезу Н0


Видим, что симпатия критиков склоняется в сторону игр на ПК (отрицательный знак критерия).

Вывод: игры на ПК критикам нравятся больше игр на PS 4.

### 3)

In [92]:
# Стрелялки
shooters = game_sales[game_sales.Genre == 'Shooter']
display(shooters.head())
print(shooters.shape)

# Стратегии
strategies = game_sales[game_sales.Genre == 'Strategy']
display(strategies.head())
print(strategies.shape)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
9,Duck Hunt,NES,1984.0,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31,NaN,NaN,NaN,NaN,NaN,NaN
29,Call of Duty: Modern Warfare 3,X360,2011.0,Shooter,Activision,9.04,4.24,0.13,1.32,14.73,88.0,81.0,3.4,8713.0,"Infinity Ward, Sledgehammer Games",M
31,Call of Duty: Black Ops 3,PS4,2015.0,Shooter,Activision,6.03,5.86,0.36,2.38,14.63,NaN,NaN,NaN,NaN,NaN,NaN
32,Call of Duty: Black Ops,X360,2010.0,Shooter,Activision,9.70,3.68,0.11,1.13,14.61,87.0,89.0,6.3,1454.0,Treyarch,M
34,Call of Duty: Black Ops II,PS3,2012.0,Shooter,Activision,4.99,5.73,0.65,2.42,13.79,83.0,21.0,5.3,922.0,Treyarch,M


(1323, 16)


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
168,Pokemon Stadium,N64,1999.0,Strategy,Nintendo,3.18,1.24,0.94,0.09,5.45,NaN,NaN,NaN,NaN,NaN,NaN
206,Warzone 2100,PS,1999.0,Strategy,Eidos Interactive,2.79,1.89,0.00,0.33,5.01,NaN,NaN,NaN,NaN,NaN,NaN
218,StarCraft II: Wings of Liberty,PC,2010.0,Strategy,Activision,2.57,1.68,0.00,0.58,4.84,93.0,82.0,8.2,3052.0,Blizzard Entertainment,T
273,Warcraft II: Tides of Darkness,PC,1995.0,Strategy,Activision,1.70,2.27,0.00,0.23,4.21,NaN,NaN,NaN,NaN,NaN,NaN
339,Pokémon Trading Card Game,GB,1998.0,Strategy,Nintendo,1.49,0.73,1.38,0.10,3.70,NaN,NaN,NaN,NaN,NaN,NaN


(683, 16)


Гипотезы и интерпретация результата соотвествуют записи в п.2. Значимость та же.

In [93]:
alpha = 0.01

In [94]:
t, p = st.ttest_ind(shooters['Critic_Score'].dropna(), strategies['Critic_Score'].dropna(), equal_var=False)
print('t-критерий:', t)
print('p-value:', p)
if p < alpha:
    print('Принимаем гипотезу Н1, отвергаем гипотезу Н0')
else:
    print('Не отвергаем гипотезу Н0')

t-критерий: -2.2972408230640315
p-value: 0.021938989522304823
Не отвергаем гипотезу Н0


В данном случае мы не можем судить о том, что критики предпочитают больше. Конечно можно задать уровень значимости 5% и тогда перевес будет в сторону стратегий, но я читал на каком-то сайте, что при количестве данных n > 100 уровень значимости обычно берут равным 1%. Таким образом, можно сделать вывод, что отношение критиков к играм в данном жанре примерно одинаково. Для более конкретного вывода можно добавить больше данных или же просто поднять уровень значимости до 5%.

# Задание 2.

In [74]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [95]:
def process_msg(message):
    """ Обрабатывает сообщение и возвращает результат. """
    
    # Приведём к нижнему регистру.
    msg_lc = message.lower()
    
    # Удаляем ненужные символы.
    only_words_msg = re.sub(r'[\W_]+', ' ', msg_lc)
    
    # Удаляем стоп-слова.
    msg_splited = only_words_msg.split()
    stopwords_set = set(stopwords.words('english'))
    without_sw = [word for word in msg_splited if word not in stopwords_set]
    
    # Выполняем лемматизацию.
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_msg = [wordnet_lemmatizer.lemmatize(word) for word in without_sw]
    
    return ' '.join(lemmatized_msg)
    
    
def vectorize_msg():
    """ Приводит сообщения к векторному виду. """
    
    tfidf = TfidfVectorizer(lowercase=False)
    tfidf_matrix = tfidf.fit_transform(df['Processed_message'])
    names = tfidf.get_feature_names()
    tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
    return tfidf_matrix

In [96]:
df = pd.read_csv('spam.csv', na_values='NaN')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [97]:
df['Processed_message'] = df.Message.apply(process_msg)
df.head()

,Category,Message,Processed_message
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think go usf life around though


In [99]:
vectorized_messages = vectorize_msg()
vectorized_messages.head()

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Выполним разделение данных и построим модель логистической регрессии.

В качестве независимой переменной скормим векторы, а в качестве зависимой - категории из датафрейма. Далее модель сопоставит векторы и категории и обучится категоризировать тестовые векторы.

In [100]:
x_train, x_test, y_train, y_test = train_test_split(vectorized_messages, df.Category, test_size=0.3, random_state=42)

In [114]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

In [106]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

LogisticRegression()

In [143]:
y_pred = lr.predict(x_test)
y_pred

array(['ham', 'ham', 'ham', ..., 'ham', 'spam', 'ham'], dtype=object)

In [153]:
accuracy_score(y_test, y_pred)

0.958732057416268

### Confusion matrix.

In [120]:
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat

array([[1445,    3],
       [  66,  158]], dtype=int64)

По матрице видим следующее: 1445+158 классов предсказано правильно, 66+3 класса предсказано неправильно.

### Датафрейм с предсказанными неправильно классами.

In [173]:
df_ = pd.concat([df[df.index.isin(y_test.index)],
                 pd.Series(y_pred, name='Predicted_category', index=y_test.index)], axis=1)
df_wrong_pred = df_[df_.Category != df_['Predicted_category']]

In [174]:
df_wrong_pred

,Category,Message,Processed_message,Predicted_category
15,spam,"XXXMobileMovieClub: To use your credit, click ...",xxxmobilemovieclub use credit click wap link n...,ham
19,spam,England v Macedonia - dont miss the goals/team...,england v macedonia dont miss goal team news t...,ham
68,spam,"Did you hear about the new ""Divorce Barbie""? I...",hear new divorce barbie come ken stuff,ham
95,spam,Your free ringtone is waiting to be collected....,free ringtone waiting collected simply text pa...,ham
135,spam,Want 2 get laid tonight? Want real Dogging loc...,want 2 get laid tonight want real dogging loca...,ham
...,...,...,...,...
4652,spam,A £400 XMAS REWARD IS WAITING FOR YOU! Our com...,400 xmas reward waiting computer randomly pick...,ham
5037,spam,You won't believe it but it's true. It's Incre...,believe true incredible txts reply g learn tru...,ham
5120,spam,PRIVATE! Your 2003 Account Statement for 078,private 2003 account statement 078,ham
5366,spam,A £400 XMAS REWARD IS WAITING FOR YOU! Our com...,400 xmas reward waiting computer randomly pick...,ham
